# Reference
`https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html`

In [1]:
# Imports
import findspark
findspark.init()
findspark.find()


'/home/prod/spark-3.3.1-bin-hadoop3'

In [2]:
from pyspark import SparkConf
conf = SparkConf()
conf.setAppName('Jupyter Structured Stream PySpark')
conf.setMaster('spark://spark-master:7077');

In [3]:
# Setup spark environment
from pyspark import SparkContext
from pyspark.sql.types import *
from pprint import pprint, pformat
sc = SparkContext.getOrCreate(conf)
print('Spark web UI link: ', sc._jsc.sc().uiWebUrl().get())


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/07 14:55:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Spark web UI link:  http://spark-master:4040


## Streaming

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

In [5]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

dStream = spark.readStream \
  .format("socket")\
  .option("host", "spark-master")\
  .option("port", 9999)\
  .load()

lines = dStream.select(
  explode(
    split(dStream.value, '\n')
  ).alias('text')
)

22/11/07 14:55:38 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [6]:
query = lines \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

22/11/07 14:55:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ae0fbd35-f670-4193-a11f-4ac6d5ec7f61. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/07 14:55:49 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+
|text|
+----+
+----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|Animal Crossing; ...|
|With a game this ...|
|Above all else, A...|
|Nintendo's comfor...|
|Animal Crossing: ...|
|Animal Crossing: ...|
|Animal Crossing: ...|
|Animal Crossing: ...|
|Animal Crossing: ...|
|I always knew Ani...|
|Animal Crossing: ...|
|The amazing, rela...|
|Animal Crossing: ...|
|New Horizons has ...|
|Animal Crossing: ...|
|Nintendo Switch g...|
|Animal Crossing i...|
|Animal Crossing f...|
|There continues t...|
|Under the illusio...|
+--------------------+
only showing top 20 rows

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|Animal Crossing: ...|
|Animal Crossing: ...|
|Nintendo's island...|
|While New Horizon...|
|Based on what I h...|
|Animal Crossing d...|
|It’s a blissfully...

-------------------------------------------
Batch: 6
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|Thanks to new cus...|
|There has never b...|
|There’s no arguin...|
|New Horizons is a...|
+--------------------+

-------------------------------------------
Batch: 7
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|We all need games...|
|Animal Crossing: ...|
|To play New Horiz...|
+--------------------+

-------------------------------------------
Batch: 8
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|Quotation forthco...|
|A perfect entry-p...|
|Time apart can be...|
+--------------------+

-------------------------------------------
Batch: 9
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|With an addictive...|
|Animal Cro

-------------------------------------------
Batch: 19
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|Animal Crossing N...|
|Animal Crossing: ...|
+--------------------+

-------------------------------------------
Batch: 20
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|New Horizons is a...|
|By consolidating ...|
|Animal Crossing N...|
|As much as I love...|
|Animal Crossing h...|
+--------------------+

-------------------------------------------
Batch: 21
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|Animal Crossing: ...|
|I can't wait to s...|
|New Horizons is s...|
|Animal Crossing: ...|
+--------------------+

22/11/07 14:57:14 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
22/11/07 14:57:14 ERROR TaskSchedulerImpl: Los

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/prod/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/prod/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/prod/anaconda3/envs/sparkimental/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
#DONE
query.stop()